In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
raw_data = pd.read_csv("C:/Users/boyerm/Documents/Thinkful/epi_r.csv")

In [3]:
raw_data.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
raw_data.sum()

title                    Lentil, Apple, and Turkey Wrap Boudin Blanc Te...
rating                                                             74482.5
calories                                                       1.00756e+08
protein                                                        1.59156e+06
fat                                                             5.5046e+06
sodium                                                         9.91985e+07
#cakeweek                                                                6
#wasteless                                                               1
22-minute meals                                                         17
3-ingredient recipes                                                    27
30 days of groceries                                                     7
advance prep required                                                  109
alabama                                                                  3
alaska                   

In [5]:
# Clean up data, drop the fields with a lot of missing data and create a binary classifier

X = raw_data.drop(['rating', 'title', 'calories', 'protein', 'fat', 'sodium'], 1)

Y = np.where(raw_data.rating <= .5,"LT_0.5",np.where(raw_data.rating <= 1.5,"0.75_to_1.5",np.where(raw_data.rating<=2.5,"1.75_to_2.5",np.where(raw_data.rating <=3.5,"2.75_to_3.5",np.where(raw_data.rating <=4.5,"3.75_to_4.5","4.75+")))))
raw_data['bin_rating'] =  np.where(raw_data.rating <= 4,0,1)
raw_datad = raw_data.drop(['rating', 'title', 'calories', 'protein', 'fat', 'sodium'], 1)

In [6]:
# Define the training and test sizes.
trainsize = int(raw_datad.shape[0] / 2)
df_test = raw_datad.iloc[trainsize:, :].copy()
df_train = raw_datad.iloc[:trainsize, :].copy()

y_train = df_train['bin_rating'].values.reshape(-1,1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['bin_rating'])]

Y_test = df_test['bin_rating'].values.reshape(-1,1)
X_test = df_test.loc[:, ~(df_train.columns).isin(['bin_rating'])]

## Use LASSO regression to reduce the  # of features

In [7]:
from sklearn.linear_model import LogisticRegression
llr = LogisticRegression(penalty='l1',C=0.045)  #lasso      # I used 0.045 to get only 30 var w/coeff <> 0
llr.fit(X_train, y_train)
print(llr.score(X_train, y_train))
lasso_params = llr.coef_
print(lasso_params)
print('\nR-squared  - Lasso:')
print(llr.score(X_test, Y_test))

0.579393576701
[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.20192667  0.          0.          0.          0.
   0.          0.          0.          0.          0.06636205  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.33168409  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        

C:\Users\boyerm\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
## This will tell me how many coefficients remain

coefficients = pd.concat([pd.DataFrame(X_train.columns),pd.DataFrame(np.transpose(llr.coef_))], axis = 1)
coefficients.columns = ['ind_var','coefficient']
good = coefficients[(coefficients['coefficient'] != 0)]

good.describe()


,coefficient
count,30.000000
mean,0.014363
std,0.176907
min,-0.435983
25%,-0.072909
50%,0.020559
75%,0.093448
max,0.367082


In [9]:
print(good)

               ind_var  coefficient
30        backyard bbq     0.201927
39                beef     0.066362
57         bon appétit     0.331684
116            chicken    -0.121579
124          christmas     0.098026
134     cocktail party    -0.012603
171         dairy free    -0.028023
179             dinner     0.102279
186              drink    -0.435983
204               fall     0.049924
228              fruit     0.021079
237        goat cheese     0.072024
240            gourmet     0.079713
251     grill/barbecue     0.247990
277     house & garden    -0.264654
306    kidney friendly     0.011400
343            low fat    -0.247564
446              pasta    -0.221086
453        peanut free     0.164773
457             pepper     0.006583
502       quick & easy    -0.021277
520              roast     0.232638
543            sausage     0.057351
560               side    -0.149684
597             summer     0.020039
616       thanksgiving     0.367082
621             tomato     0

In [10]:
x_columns = good['ind_var']
y_column = ['bin_rating']

## try KNN for this project

In [11]:
from sklearn.neighbors import KNeighborsClassifier
# Create the knn model. Look at the five closest neighbors.
knn = KNeighborsClassifier(n_neighbors=5,weights = 'distance')

In [12]:
# Fit the model on the training data.

knn.fit(X_train[x_columns], y_train.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')

In [13]:
# Make point predictions on the test set using the fit model.
predictions = knn.predict(X_test[x_columns])

# Get the actual values for the test set.
actual = Y_test

In [14]:
print(pd.crosstab(predictions.ravel(), actual.ravel()))

col_0     0     1
row_0            
0      2424  2183
1      2271  3148


In [15]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(knn,X_train[x_columns],y_train.ravel(), cv=5)
print(score)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

[ 0.55284148  0.56430708  0.54114713  0.56259352  0.54391218]
Unweighted Accuracy: 0.55 (+/- 0.02)


## TRY SVM Classifier

In [16]:

from sklearn.svm import SVC
# this takes a while
clf = SVC()
clf.fit(X_train[x_columns], y_train.ravel()) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
# Make point predictions on the test set using the fit model.
predictions = clf.predict(X_test[x_columns])

# Get the actual values for the test set.
actual = Y_test

In [18]:
print(pd.crosstab(predictions.ravel(), actual.ravel()))

col_0     0     1
row_0            
0      1502  1005
1      3193  4326


In [19]:
score = cross_val_score(clf,X_train[x_columns],y_train.ravel(), cv=5)

In [20]:
print(score)

[ 0.5772682   0.57676969  0.58204489  0.58553616  0.5748503 ]


In [21]:
print("SVM Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

SVM Accuracy: 0.58 (+/- 0.01)


## ugh, not much better!

### what about SVC against all columnns?

In [23]:
clf.fit(X_train, y_train.ravel()) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
# Make point predictions on the test set using the fit model.
predictions = clf.predict(X_test)

# Get the actual values for the test set.
actual = Y_test

In [25]:
print(pd.crosstab(predictions.ravel(), actual.ravel()))

col_0     0     1
row_0            
0       303   152
1      4392  5179


In [26]:
score = cross_val_score(clf,X_train,y_train.ravel(), cv=5)